# Zero-Shot Text Classification

<p>Workflow test di classificazione testo Zero-Shot.</p>
Dato il numero di elementi da verificare, eseguirà la classificazione su: </br>
    - Riscontro guasti; </br>
    - Area di classificazione del guasto; </br>
    - Azione intrapresa

In [3]:
import pandas as pd
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Load dataset
df = pd.read_excel("docs/ClosingNotes - Copy - Copy.xlsx")

# Inspect the first few rows
print(df.head())

                                        Closing Note  Selected Selected.1  \
0  Sostituito cassetto da 50 e configurato. Il ca...  CASSETTE        UNK   
1  Cassetto in stato inoperativo dopo inceppament...  CASSETTE        UNK   
2      Sostituzione cassetti DC fit e unfit. Test ok  CASSETTE        UNK   
3  Trovato ATM con fuori servizio su dispensatore...  CASSETTE        UNK   
4  Gr: ATM trovato in servizio. Trovate banconote...  CASSETTE        UNK   

     Action  Action.1                  Guasto  
0  REPLACED  REPLACED      Guasto riscontrato  
1  REPLACED  REPLACED      Guasto riscontrato  
2  REPLACED  REPLACED      Guasto riscontrato  
3   CLEANED   CLEANED      Guasto riscontrato  
4  REPLACED  REPLACED  Guasto non riscontrato  


In [4]:
# Filter out rows without review text
df = df.dropna(subset=["Closing Note"]).reset_index(drop=True)

# Extract the review texts into X
X = df["Closing Note"].tolist()

dff = pd.read_excel("docs/ReportFeedbacks_2025-05-12_11-25.xlsx")
dff = dff.dropna(subset=["Closing Note"]).reset_index(drop=True)
Z = dff["Closing Note"].head(10).tolist()


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Transform the text data to feature vectors
X = vectorizer.fit_transform(df["Closing Note"])

# Labels
y = np.asarray(df[df.columns[1:]])

#y = df['Selected']

In [6]:
CT = ["Cash Transport", "CT", "piatti", "LT", "ST", "PT"]
NE = ["Note Escrow", "NE", "nastro", "PE", "leva 10", "ME", "precas", "escrow"]
NF = ["Note Feeder", "NF", "bocchetta", "leva 1", "sfogliatore", "SF", "PF", "MF", "stacker", "upper unit"]
NV = ["Note Validator", "NV", "leva 7", "n validator", "SNV", "UNV", "thickness", "Feeder"]
CASSETTE = ["Cassetto", "Cassetti", "RC", "AC", "DC", "MC"]
SHUTTER = ["Shutter"]

areas = CT + NE + NF + NV + CASSETTE + SHUTTER + ["None"]

NO_ACTION = ["nessuna azione"]
REPAIRED = ["riparazione", "riposizionat"]
REPLACED = ["sostituzione"]
CALIBRATED = ["calibrazione"]
CLEANED = ["pulizia", "rimozione"]

operations = NO_ACTION + REPAIRED + REPLACED + CALIBRATED + CLEANED

# Label da sistemare, anche il prompt poichè non affidabile ancora
state = ["Guasto riscontrato", "fuori servizio", "Guasto non riscontrato", "sconosciuto"]

In [7]:
from pydantic import BaseModel

class Classification(BaseModel):
    part: str

In [8]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from skollama.models.ollama.classification.few_shot import MultiLabelFewShotOllamaClassifier, DynamicFewShotOllamaClassifier
from skllm.datasets import get_multilabel_classification_dataset
from skollama.models.ollama.classification.zero_shot import CoTOllamaClassifier

#y = [
#    "CASSETTE", "CASSETTE", "CASSETTE", "CASSETTE", "CASSETTE",
#    "CT", "CT", "CT", "CT", "CT",
#    "NE", "NE", "NE", "NE", "NE",
#    "NF", "NF", "NF", "NF", "NF",
#    "NV", "NV", "NV", "NV", "NV",
#    "SHUTTER", "SHUTTER", "SHUTTER", "SHUTTER", "SHUTTER"
#]

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Initialize and train the classifier
clf = MultiOutputClassifier(KNeighborsClassifier(5))
    #GaussianNB())
    #AdaBoostClassifier(random_state=42))
    #DecisionTreeClassifier(max_depth=25, random_state=23),)
clf.fit(X_train.toarray(), y_train)

y_pred = clf.predict(X_test.toarray())
print(y_pred)

# Evaluate the performance
#accuracy = accuracy_score(y_test, y_pred)
#report = classification_report(y_test, y_pred, zero_division=0.0, labels=areas)

#print(f'Accuracy: {accuracy:.4f}')
#print('Classification Report:')
#print(report)

[['NV' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']
 ['SHUTTER' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']
 ['CT' 'UNK' 'CLEANED' 'CLEANED' 'Guasto riscontrato']
 ['SHUTTER' 'UNK' 'CLEANED' 'CLEANED' 'Guasto riscontrato']
 ['NV' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']
 ['CASSETTE' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']
 ['NV' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']]


In [9]:
def predict_category(text):
    """
    Predict the category of a given text using the trained classifier.
    """
    text_vec = vectorizer.transform([text])
    prediction = clf.predict(text_vec.toarray())
    return prediction

from sklearn.datasets import fetch_20newsgroups


# Example usage
sample_text = """
Sostituzione cash transport a causa di una cinghia rotta.Prove ok.
"""
predicted_category = predict_category(sample_text)
print(f'The predicted categories are: {predicted_category}')

The predicted categories are: [['CT' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']]


In [10]:
import numpy as np

df = pd.read_excel("docs/ReportFeedbacks_2025-05-12_11-25.xlsx", index_col=None, na_values=['NA'])
df.replace(np.nan, 0, inplace=True)

count = 1
for row_name, row in df.iterrows():
    st1 = row['Closing Note']
    if str(st1) == "0":
        continue
    predicted_category = predict_category(str(st1))
    print(st1)
    print(f'The predicted categories are: {predicted_category}')
    count += 1
    if count > 20:
        break

MTA in regolare servizio.
Effettuato controllo visivo, e controllo LOG.
Effettuate diverse prove di dispensazione e versamento tramite tool.
Funzionamento ok.
The predicted categories are: [['NV' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']]
MTA IN SERVIZIO. 
Anomalia risolta in autonomia dalla filiale.
Effettuate verifiche hardware e controllo LOG.
Funzionalità ok.
The predicted categories are: [['NV' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']]
MTA FS.
Effettuato controllo log e controllo visivo, rinvenute bnc in zona NV e NE, inoltre riposizionate cinghie NE fuori sede.
Funzionalità ripristinate prove effettuate con clienti intesa san paolo.
The predicted categories are: [['NV' 'UNK' 'REPLACED' 'REPLACED' 'Guasto riscontrato']]
Riscontrato mta fs.
Effettuati controllo log e controllo visivo dispensatore,  riscontrato pezzo di carta in zona SNV che causava lo scarto delle bnc.
Riscontrata inoltre moneta in zona NF, rimossa ed eseguite prove di funzionamento con tool.
MTA F